# 9.7 Cleaning Network-Constrained Trajectories
Perform map-matching using a Valhalla server 

In [7]:
#%% IMPORT LIBRARIES
import folium
import pandas as pd
import geopandas as gpd
import json
import requests
from shapely.geometry.linestring import LineString
from decode_functions import decode
from shapely.geometry import Point

In [8]:
#%% READ & FORMAT GPS INFO
csv_file = "ToIxelles.csv"
df_rawGPS = pd.read_csv(csv_file)

# Ensure the CSV has 'lon', 'lat', and 'timestamp' columns
if 'Longitude' not in df_rawGPS.columns or 'Latitude' not in df_rawGPS.columns or 'Timestamp' not in df_rawGPS.columns:
    raise ValueError("The CSV file must contain 'lon', 'lat', and 'timestamp' columns.")

# Convert 'timestamp' column to datetime format
df_rawGPS['Timestamp'] = pd.to_datetime(df_rawGPS['Timestamp'])

# Convert the 'timestamp' to epoch seconds
df_rawGPS['epoch_seconds'] = df_rawGPS['Timestamp'].astype('int64') // 10**9

df_rawGPS = df_rawGPS.rename(columns={'Latitude': 'lat', 'Longitude': 'lon', 'epoch_seconds': 'time'})

# Create GeoDataFrame
gdf_rawGPS = gpd.GeoDataFrame(df_rawGPS, 
                              geometry=gpd.points_from_xy(df_rawGPS.lon, df_rawGPS.lat), 
                              crs="EPSG:4326")

In [9]:
gdf_rawGPS

,Timestamp,lon,lat,Altitude,Speed,Course,time,geometry
0,2024-04-16 07:20:08+00:00,4.304368,50.788045,49.04,0.15,0.00,1713252008,POINT (4.30437 50.78805)
1,2024-04-16 07:20:09+00:00,4.304189,50.788019,48.32,0.17,0.00,1713252009,POINT (4.30419 50.78802)
2,2024-04-16 07:20:33+00:00,4.304284,50.788152,57.39,8.17,0.10,1713252033,POINT (4.30428 50.78815)
3,2024-04-16 07:20:35+00:00,4.304313,50.788300,58.38,7.55,0.11,1713252035,POINT (4.30431 50.7883)
4,2024-04-16 07:20:37+00:00,4.304331,50.788416,60.71,5.32,0.11,1713252037,POINT (4.30433 50.78842)
...,...,...,...,...,...,...,...,...
572,2024-04-16 07:50:43+00:00,4.377803,50.813532,137.33,9.99,2.69,1713253843,POINT (4.3778 50.81353)
573,2024-04-16 07:50:44+00:00,4.377893,50.813413,138.14,10.81,2.66,1713253844,POINT (4.37789 50.81341)
574,2024-04-16 07:50:45+00:00,4.377993,50.813310,138.60,11.67,2.58,1713253845,POINT (4.37799 50.81331)
575,2024-04-16 07:50:46+00:00,4.378076,50.813215,138.78,11.81,2.56,1713253846,POINT (4.37808 50.81321)


In [10]:
 #%% VALHALLA REQUEST
meili_coordinates = df_rawGPS.to_json(orient='records')
meili_head = '{"shape":'
meili_tail = ""","search_radius": 10, "shape_match":"map_snap", "costing":"auto", "use_timestamps": true, "format":"osrm"}"""
meili_request_body = meili_head + meili_coordinates + meili_tail
url = "https://valhalla1.openstreetmap.de/trace_route"
headers = {'Content-type': 'application/json'}
data = str(meili_request_body)
r = requests.post(url, data=data, headers=headers)


#%% READ & FORMAT VALHALLA RESPONSE
if r.status_code == 200:
    response_text = r.json()#loads(r.text)
    search_1 = response_text.get('matchings')
    search_2 = dict(search_1[0])
    polyline6 = search_2.get('geometry')
    search_3 = response_text.get('tracepoints')

In [11]:
# Decode the polyline and create a LineString object
lst_MapMatchingRoute = LineString(decode(polyline6))

# Create a GeoDataFrame for the map-matching route
gdf_MapMatchingRoute = gpd.GeoDataFrame(
    geometry=[lst_MapMatchingRoute], crs="EPSG:4326")

# Extract the individual points from the LineString
gdf_MapMatchingRoute_points = gdf_MapMatchingRoute.explode(index_parts=False)

# Create GeoDataFrame for map-matched GPS points
df_mapmatchedGPS_points = pd.DataFrame(
    [d['location'] for d in search_3 if d and 'location' in d], 
    columns=['lon', 'lat'])
gdf_mapmatchedGPS_points = gpd.GeoDataFrame(
    df_mapmatchedGPS_points, geometry=gpd.points_from_xy(
        df_mapmatchedGPS_points['lon'], df_mapmatchedGPS_points['lat']), 
    crs="EPSG:4326")

# Drop json-unserializable columns from raw GPS GeoDataFrame
gdf_rawGPS = gdf_rawGPS.drop(columns=['Timestamp'])

# Create a LineString from the raw GPS points in gdf_rawGPS
raw_gps_line = LineString(gdf_rawGPS.geometry.apply(lambda point: (point.x, point.y)))

# Create a GeoDataFrame for the raw GPS trajectory LineString
gdf_RawGPSTrajectory = gpd.GeoDataFrame(geometry=[raw_gps_line], crs="EPSG:4326")

# Initialize the map - raster tiles
#m = folium.Map(location=[50.802545, 4.341914], tiles='openstreetmap', zoom_start=14)

# Initialize the map - vector tiles
m = folium.Map(location=[50.802545, 4.341914], tiles=None, zoom_start=14)

# Add OpenStreetMap vector tiles as a base layer
folium.TileLayer(
    tiles="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
    attr="OpenStreetMap",
    name="OSM Vector Tiles",
    overlay=False,
    control=True
).add_to(m)

# Add raw GPS points to the map
folium.GeoJson(gdf_rawGPS, style_function=lambda x: {'color': 'blue'}, 
    marker=folium.CircleMarker(radius=4, weight=0, fill_color='blue', fill_opacity=1), 
    name='Raw GPS Points (blue)').add_to(m)

# Add the original raw GPS trajectory as a LineString on the map
folium.GeoJson(
    gdf_RawGPSTrajectory,
    style_function=lambda x: {'color': 'blue', 'weight': 4, 'opacity': 1},
    name='Raw GPS Trajectory (blue)'
).add_to(m)

# Add map-matched GPS points to the map
folium.GeoJson(gdf_mapmatchedGPS_points, marker=folium.CircleMarker(
    radius=4, weight=0, fill_color='red', fill_opacity=1), 
    name='Map-Matched GPS Points (red)').add_to(m)

# Add map-matching route to the map
folium.GeoJson(gdf_MapMatchingRoute,     
    style_function=lambda x: {'color': 'red', 'weight': 4, 'opacity': 1},
    name='Map Matched trajectory (red)').add_to(m)


# Add layer control to switch between different layers
folium.LayerControl(position='topright', collapsed=False).add_to(m)

# Save to html then export to pdf for the book quality
m.save("mapmatching.html")

# Display the map in the notebook
m